In [35]:
from delta_rest_client import DeltaRestClient, OrderType
import json
import sys
import requests
from time import time
import hmac
import hashlib
from utils import *
import pytz
from indicators import Indicators

key = None
secret = None
with open("creds.json",'r') as f:
    op = json.load(f)
    key = op.get('api')
    secret = op.get('secret')

if key is None or secret is None:
    print(key)
    print(secret)
    sys.exit(1)
baseurl = "https://api.india.delta.exchange"

client = DeltaRestClient(
  base_url=baseurl,
  api_key=key,
  api_secret=secret
)
productId = get_product_id_by_symbol(symbol = 'BTCUSD')
client.set_leverage(productId, 30)

{'leverage': '30', 'order_margin': '0', 'product_id': 27}

In [36]:
class symbol_management:
    def __init__(self, client, productId):
        self.productId = productId
        self.client = client
        self.pnl = 0
        self.minqty = 3
        self.tradesize = self.minqty

    def recordorder(self, entry, stoploss, qty, target, tradetime, tradetype, openorderid):
        data = None
        with open("trades.json", "r") as f:
            data = json.load(f)
        if data.get(tradetime) is not None:
            return 
        data['trade']={
            "tradetime" : tradetime,
            "entry": entry,
            "stoploss": stoploss,
            "target": target,
            "tradetype": tradetype,
            "qty" : qty,
            "openorderid" : openorderid,
        }
        with open("trades.json", "w") as f:
            json.dump(data, f, indent=2)
            print("saved data")
        
    
    def trade(self, entry, stoploss, tradetype, target, tradetime):
        qty = self.minqty
        if self.pnl < 0:
            qty = self.tradesize + 1
        if self.pnl > 0:
            qty = min(self.minqty,self.tradesize - 1)
        order = self._place_order(tradetype, entry, qty)
        if order is not None:
            self.recordorder(entry, stoploss, qty, target, tradetime, tradetype, order['id'])
       

    def _manage_active_order(self):
        activesize = self.client.get_position(self.productId)['size']
        activesize = abs(activesize)
        if abs(activesize) != 0:  
            trades = None
            with open("trades.json", "r") as f:
                trades = json.load(f)
            
            latest_trade = trades.get('trade')
            if latest_trade is None:
                print("Warning : This is not possible as there is an active order and there is no track of that order in your json file")
                return True
            
            r =  client.get_live_orders()
            stoplossorderid = None
            targetorderid = None

            for orders in r:
                if  orders['state'] == 'open' and orders['product_id'] == self.productId and orders['client_order_id']=='take_profit' :
                    targetorderid = orders['id']
                elif orders['stop_order_type'] == 'stop_loss_order' and orders['state'] == 'pending' and orders['product_id'] == self.productId:
                    stoplossorderid = orders['id']
            # print("stoplossorderis :" , stoplossorderid)
            # print("targetorderid : " , targetorderid)
            
            if stoplossorderid is None:
                print("missing stoploss order so placing a new one ...")
                self.validate_stoploss_order(latest_trade['stoploss'], latest_trade['tradetype'], activesize)

            if targetorderid is None:
                print("missing target order so placing a new one ...")
                self.validate_target_order(latest_trade['target'], latest_trade['tradetype'], activesize)
            return True
        return False

    def _cancel_all_stoploss_target_orders(self):
        r =  client.get_live_orders()
        for orders in r:
            if orders['state'] == 'pending' and orders['product_id'] == self.productId:
                print("cancelling stoploss orders from state management")
                self._cancel_order(orders['id'])
            if orders['state'] == 'open' and orders['product_id'] == self.productId and orders['client_order_id']=='take_profit' :
                print("cancelling target orders from state management")
                self._cancel_order(orders['id'])

    def _state_manage(self):
        try:
            if not self._manage_active_order():
                self._cancel_all_stoploss_target_orders()
        except Exception as e:
            print("Error in state management : ", str(e))

                      
    def _is_order_open(self, id):
        r =  client.get_live_orders()
        for orders in r:
            if  orders['order_type']=='limit_order' and orders['state'] == 'open' and orders['product_id'] == self.productId and orders['id'] == id:
                return True
        return False
    
    def _cancel_order(self, orderid):
        self.client.cancel_order(productId, orderid)
    
    def print_position_state(self):
        print("product id : ", self.productId)
        print("isactiveposition : ", self.isactiveposition)
        print("stoplossid : ", self.stoplossid)
        print("targetid : ",self.targetid)
        print("opentrade : ", self.opentradeid)
        print("myposition state : ", self.activeTradeState) 
    
    def _is_safe_placing(self):
        activesize = self.client.get_position(self.productId)['size']
        activesize = abs(activesize)
        if activesize > 0:
            print("NOSAFE to place order as asctive position found")
            return False
        r =  client.get_live_orders()
        print('cancelling orders before plaing trade')
        for orders in r:
            if  (orders['state'] == 'pending' or orders['state'] == 'open') and orders['product_id'] == self.productId:
                self._cancel_order(orders['id'])

        r =  client.get_live_orders()
        for orders in r:
            if  (orders['state'] == 'pending' or orders['state'] == 'open') and orders['product_id'] == self.productId:
                print("NOSAFE as found a trade so cacelling this order")
                return False
        return True        
                

    def _place_order(self, tradetype, entry, qty):
        if self._is_safe_placing():
            order = self.client.place_order(self.productId, qty, tradetype, limit_price=entry, time_in_force=None, order_type=OrderType.LIMIT)
            print("Limit order placed : " , order)
            return order
        return None
        
    def validate_stoploss_order(self, stoploss,tradetype, qty):
        stop_order = self.client.place_stop_order(
        product_id=self.productId,
        size=qty,
        side='sell' if tradetype == 'buy' else 'buy',
        stop_price=stoploss,
        limit_price=stoploss,
        order_type=OrderType.MARKET,
        isTrailingStopLoss=False
        )
        return stop_order['id']
        
    def validate_target_order(self, target,tradetype, qty):    
        target_order = self.client.place_order(
        product_id=self.productId,
        size=qty,
        side='sell' if tradetype == 'buy' else 'buy',
        limit_price=target,
        order_type=OrderType.LIMIT,
        client_order_id="take_profit"
        )

        return target_order['id']

    def update_trade_state(self):
        r = self.client.get_margined_position(self.productId)
        if r is not None:
            self.pnl = float(r['unrealized_pnl'])
            self.tradesize = abs(int(r['size']))

In [ ]:
import requests
import time
from datetime import datetime, timedelta
import pandas as pd
import pytz
symbolmanager = symbol_management(client, productId)

class Strategy:
    def __init__(self, symbol, rr = 2.5, timer = 1, interval = '3m', tick = 3):
        self.symbol = symbol
        self.rr = rr
        self.timer = timer
        self.interval = interval

    def _get_unix_timestamp(self, dt):
        return int(time.mktime(dt.timetuple()))

    def _load_historical_data(self,days=1):
        interval = self.interval
        end_dt = datetime.now() + timedelta(minutes=1)
        start_dt = end_dt - timedelta(days=days)

        start_unix = self._get_unix_timestamp(start_dt)
        end_unix = self._get_unix_timestamp(end_dt)

        url = 'https://api.india.delta.exchange/v2/history/candles'
        params = {
            'resolution': interval,
            'symbol': self.symbol,
            'start': start_unix,
            'end': end_unix
        }
        headers = {'Accept': 'application/json'}
        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['result'])
            df['time'] = pd.to_datetime(df['time'], unit='s',  utc=True).dt.tz_convert(pytz.timezone('Asia/Kolkata'))
            df = df[::-1]
            return df
        else:
            raise ConnectionError(f"Failed to fetch data: {response.status_code} - {response.text}")

    def load_data(self):
        try:
            df = self._load_historical_data()
            # df = Indicators.supertrend(df, 1,1, 'supertrend')
            # df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
            # df['buy'] = (df['supertrend_1_1_direction'].shift(1)==-1) & (df['supertrend_1_1_direction']==1)
            # df['sell'] = (df['supertrend_1_1_direction'].shift(1)==1) & (df['supertrend_1_1_direction']==-1)
            df = Indicators.ema(df, 11, 'ema')
            df['buy'] = (df['close'] > df['ema_11']) & (df['close'].shift(1) < df['ema_11'].shift(1))
            df['sell'] = (df['close'] < df['ema_11']) & (df['close'].shift(1) > df['ema_11'].shift(1))
            print(df.head(1))
            lastminute = datetime.strptime(df.iloc[-1]['time'], "%Y-%m-%d %H:%M:%S").minute
            currentminute = datetime.today().minute

            if currentminute % 3 == 0:
                df = df.tail(1)
            else:
                df = df.tail(2).head(1)

            row = df.iloc[0]
            print(df[['close', 'buy',  'sell' , 'time']])
            return df.iloc[-1]
        except Exception as e:
            print("error in dataloading", str(e))
        return None

    def run_strategy(self, row):
        # try: 
            if row['buy']:
                entry = row['close'] + 10
                stoploss = row['low'] - 25
                target = entry + self.rr * (entry - stoploss)
                entry = row['close']
                symbolmanager.trade(entry, stoploss, 'buy', target,row['time'][:16])
                return 

            if row['sell']:
                entry = row['close'] - 10
                stoploss = row['high'] + 25
                target = entry - self.rr * (stoploss - entry)
                symbolmanager.trade(entry, stoploss, 'sell', target,row['time'][:16])
                return 
           
        # except Exception as e:
            # print(f"Error running strategy: {e}")
        

    def start(self):
        lastactualminute = datetime.today().minute 
        while(True):
            time.sleep(10)
            # print(lastactualminute, datetime.today().minute)
            if lastactualminute > datetime.today().minute:
                time.sleep(3)
                symbolmanager._state_manage()
                symbolmanager.update_trade_state()
                # print("skipping")
                continue


            time.sleep(4)
            row = self.load_data()
            if row is not None:
                print("data loaded and ready to play strategy")
                # self.run_strategy(row)
                lastactualminute = datetime.today().minute
            else:
                print("Failed to load data")

strategy = Strategy('BTCUSD')
strategy.start()

        close      high       low      open                      time  volume  \
478  119578.5  119634.5  119470.5  119472.0 2025-07-17 00:24:00+05:30   12156   

     ema_11    buy   sell  
478     NaN  False  False  
error in dataloading strptime() argument 1 must be str, not Timestamp
Failed to load data
        close      high       low      open                      time  volume  \
478  119578.5  119634.5  119470.5  119472.0 2025-07-17 00:24:00+05:30   12156   

     ema_11    buy   sell  
478     NaN  False  False  
error in dataloading strptime() argument 1 must be str, not Timestamp
Failed to load data
        close      high       low      open                      time  volume  \
478  119578.5  119634.5  119470.5  119472.0 2025-07-17 00:24:00+05:30   12156   

     ema_11    buy   sell  
478     NaN  False  False  
error in dataloading strptime() argument 1 must be str, not Timestamp
Failed to load data
        close      high       low      open                      time  vo

KeyboardInterrupt: 